In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import textblob
from textblob import TextBlob, Word
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
import string
table = str.maketrans('', '', string.punctuation)
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk.stem import wordnet
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
word_lem = WordNetLemmatizer()
from textblob import TextBlob

import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def read_dataset():
    data = pd.read_csv(r"/content/drive/MyDrive/Data Set Suicidal/unprocessed.csv", sep = '\t', header = None)
    data.columns = ['sentence', 'labels']
    return data

In [ ]:
def sample_data(data, sample_value):
    data = data[:][:sample_value]
    return data

In [ ]:
print(read_dataset())

                                                 sentence  labels
0       I’m having anxiety right now and it’s not goin...       0
1       Its been a hard hard week for me, i decided to...       0
2       I was extremely suicidal 8 months ago. I reali...       1
3       After a long and terrible relationship and not...       0
4       Specifically against Yunkai and Astapor. Two c...       0
...                                                   ...     ...
207417  The love of my life is slowly drifting away fr...       1
207418  Because of all the shit that is happening in t...       1
207419  Kind of an oddball question, but for anybody w...       1
207420  The thought of taking exams is triggering suic...       1
207421  I havent been clinically/psychologically diagn...       0

[207422 rows x 2 columns]


In [ ]:
data = read_dataset()

In [ ]:
print(data)

                                                 sentence  labels
0       I’m having anxiety right now and it’s not goin...       0
1       Its been a hard hard week for me, i decided to...       0
2       I was extremely suicidal 8 months ago. I reali...       1
3       After a long and terrible relationship and not...       0
4       Specifically against Yunkai and Astapor. Two c...       0
...                                                   ...     ...
207417  The love of my life is slowly drifting away fr...       1
207418  Because of all the shit that is happening in t...       1
207419  Kind of an oddball question, but for anybody w...       1
207420  The thought of taking exams is triggering suic...       1
207421  I havent been clinically/psychologically diagn...       0

[207422 rows x 2 columns]


In [ ]:
def avg_word_len (sentence):
    words = sentence.split()
    avg_len = sum(len(word) for word in words)/len(words)
    return avg_len

def extract_ngrams(data, num):
    '''
    Function to generate n-grams from sentences
    '''
    n_grams = TextBlob(data).ngrams(num)
    return [ ' '.join(grams) for grams in n_grams]

In [ ]:
print(data.shape)
data.head()

(207422, 2)


,sentence,labels
0,I’m having anxiety right now and it’s not goin...,0
1,"Its been a hard hard week for me, i decided to...",0
2,I was extremely suicidal 8 months ago. I reali...,1
3,After a long and terrible relationship and not...,0
4,Specifically against Yunkai and Astapor. Two c...,0


In [ ]:
data['char_count'] = data['sentence'].str.len()
df_train_sort_charcount = data.sort_values(by='char_count', ascending=False)
df_train_sort_charcount[['sentence', 'char_count']].head()

,sentence,char_count
95803,https://www.evernote.com/shard/s405/sh/cb9e84c...,39883
52839,Felicia: [Part 1](https://www.reddit.com/r/nos...,39366
117650,Some may remember me. I was asked what’s going...,37055
117543,"Microwave, Penicillin, chocolate chip cookies,...",36819
91805,\[Before I post this: this turned out way long...,36788


In [ ]:
data['word_count'] = data['sentence'].apply(lambda x: len(str(x).split(" ")))
df_train_sort_wordcount = data.sort_values(by='word_count', ascending=False)
df_train_sort_wordcount[['sentence','word_count']].head()

,sentence,word_count
95803,https://www.evernote.com/shard/s405/sh/cb9e84c...,7115
117650,Some may remember me. I was asked what’s going...,7027
91805,\[Before I post this: this turned out way long...,6707
186512,I’m an avid explorer. For as long as I can rem...,6518
117543,"Microwave, Penicillin, chocolate chip cookies,...",6512


In [ ]:
data = data['sentence'][0]
print("1-gram: ", extract_ngrams(data, 1))
print("2-gram: ", extract_ngrams(data, 2))
print("3-gram: ", extract_ngrams(data, 3))
print("4-gram: ", extract_ngrams(data, 4))

1-gram:  ['I', '’', 'm', 'having', 'anxiety', 'right', 'now', 'and', 'it', '’', 's', 'not', 'going', 'away', 'it', 'didn', '’', 't', 'work', 'to', 'eat', 'I', '’', 'm', 'so', 'worried', 'about', 'getting', 'sick', 'What', 'can', 'I', 'do']
2-gram:  ['I ’', '’ m', 'm having', 'having anxiety', 'anxiety right', 'right now', 'now and', 'and it', 'it ’', '’ s', 's not', 'not going', 'going away', 'away it', 'it didn', 'didn ’', '’ t', 't work', 'work to', 'to eat', 'eat I', 'I ’', '’ m', 'm so', 'so worried', 'worried about', 'about getting', 'getting sick', 'sick What', 'What can', 'can I', 'I do']
3-gram:  ['I ’ m', '’ m having', 'm having anxiety', 'having anxiety right', 'anxiety right now', 'right now and', 'now and it', 'and it ’', 'it ’ s', '’ s not', 's not going', 'not going away', 'going away it', 'away it didn', 'it didn ’', 'didn ’ t', '’ t work', 't work to', 'work to eat', 'to eat I', 'eat I ’', 'I ’ m', '’ m so', 'm so worried', 'so worried about', 'worried about getting', '

In [ ]:
def convert_to_string (df_train):
    for i in range(len(df_train)):
        df_train['sentence'][i] = str(df_train['sentence'][i])
    return data

def data_processing(data):
    for i in range(len(data)):
        # Remove Excape Sequences
        sentence = remove_excape_sequesces(data['sentence'][i])
        # Remove Links
        sentence = remove_links(sentence)
        # Forming the final paragraphs
        senetnce = form_paragraph(sentence)
        # Perform further NLP processing
        sentence = apply_NLP(sentence)

        data['sentence'][i] = sentence
    return data

def apply_NLP (sentence):
    sentences = sentence.split(sep = '.')
    for i in range(len(sentences)):
        # Tokenize the words
        tokens = word_tokenize(sentences[i])
        tokens = [word.lower() for word in tokens]
        # Remove puntuations
        no_punctuations = [word.translate(table) for word in tokens]
        # Remove all non-alphabetic characters.
        words = [word for word in no_punctuations if word.isalpha()]
        # Remove Stop words and convert the sentence to its base form
        words = [word_lem.lemmatize(w) for w in words if not w in stop_words]
        sentences[i] = " ".join(words)
    sentence = " .".join(sentences)
    return sentence+"."

def remove_excape_sequesces (sentence):
    excape_sequences = ['\n', '\t', '\r', '\\', '\a', '\f', '\o', '\v', '\b', '\"', '\'', '\newline']
    temp = ""
    sentence = str(sentence)
    for i in sentence:
        if i in excape_sequences:
            continue
        temp += i
    return temp

def remove_links (sentence):
    comment = sentence.split(sep = " ")
    for word in comment:
        if ('http' in word) or ('www' in word) or ('https' in word) or (len(word) == 0):
            comment.remove(word)
    sentence = " ".join(comment)
    return sentence

def form_paragraph (sentence):
    temp = nltk.sent_tokenize(sentence)
    for index in range(len(temp)):
        temp[index] = temp[index].replace('.', ' ')
    for index in range(len(temp)):
        d = []
        k = temp[index].split(sep = ' ')
        for z in range(len(k)):
            if len(k[z]) > 0:
                d.append(k[z]) 
        s = ''
        for j in range(len(d)):
            s = s + ' ' + d[j].strip()
        temp[index] = s.strip()      
    sentence = '. '.join(temp).lower()
    return sentence

In [ ]:
data = read_dataset()
data = sample_data(data, 200000)
data = convert_to_string(data)
data = data_processing(data)
data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,sentence,labels
0,anxiety right going away . . .work eat . . .wo...,0
1,hard hard week decided go get happy drink orde...,0
2,extremely suicidal month ago .realized ive rui...,1
3,long terrible relationship going school anxiet...,0
4,specifically yunkai astapor .two city full inn...,0
...,...,...
2595,im scared .scared .want die want die make sens...,0
2596,whatever silly mean everything monty python ba...,0
2597,frozen e menace pour le royaumela franchise de...,0
2598,connected iphone using bluetooth everything se...,0


In [ ]:
!pip install transformers==3.0.2

In [ ]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
data.shape

(2600, 2)

In [ ]:
data.head()

,sentence,labels
0,anxiety right going away . . .work eat . . .wo...,0
1,hard hard week decided go get happy drink orde...,0
2,extremely suicidal month ago .realized ive rui...,1
3,long terrible relationship going school anxiet...,0
4,specifically yunkai astapor .two city full inn...,0


In [ ]:
data['labels'].unique()

array([0, 1])

In [ ]:
data.describe()

,labels
count,2600.000000
mean,0.514231
std,0.499894
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [ ]:
new_df = data[['sentence', 'labels']]

In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
# EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

In [ ]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.sentence
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
train_size = 0.8
train_data=new_df.sample(frac=train_size,random_state=200)
test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))



FULL Dataset: (2600, 2)
TRAIN Dataset: (2080, 2)
TEST Dataset: (520, 2)


In [ ]:
training_set = SentimentData(train_data, tokenizer, MAX_LEN)
testing_set = SentimentData(test_data, tokenizer, MAX_LEN)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 5)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = RobertaClass()
model.to(device)

RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, eleme

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [ ]:
EPOCHS = 100
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]

Training Loss per 5000 steps: 1.5758798122406006
Training Accuracy per 5000 steps: 50.0


260it [01:36,  2.69it/s]
0it [00:00, ?it/s]

The Total Accuracy for Epoch 0: 66.63461538461539
Training Loss Epoch: 0.7043719624097531
Training Accuracy Epoch: 66.63461538461539
Training Loss per 5000 steps: 0.5146476030349731
Training Accuracy per 5000 steps: 87.5


260it [01:39,  2.61it/s]
0it [00:00, ?it/s]

...

The Total Accuracy for Epoch 2: 85.57692307692308
Training Loss Epoch: 0.3650114803360059
Training Accuracy Epoch: 85.57692307692308
Training Loss per 5000 steps: 0.13717471063137054
Training Accuracy per 5000 steps: 100.0


260it [01:40,  2.58it/s]
0it [00:00, ?it/s]

The Total Accuracy for Epoch 3: 88.5576923076923
Training Loss Epoch: 0.3182661325742419
Training Accuracy Epoch: 88.5576923076923
Training Loss per 5000 steps: 0.5661956071853638
Training Accuracy per 5000 steps: 75.0


260it [01:40,  2.58it/s]

The Total Accuracy for Epoch 4: 91.39423076923077
Training Loss Epoch: 0.25585295026405497
Training Accuracy Epoch: 91.39423076923077


In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu

In [ ]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

1it [00:00,  9.65it/s]

Validation Loss per 100 steps: 0.2892468273639679
Validation Accuracy per 100 steps: 75.0


130it [00:08, 15.18it/s]

Validation Loss Epoch: 0.5516286753404599
Validation Accuracy Epoch: 77.88461538461539
Accuracy on test data = 77.88%


In [ ]:
output_model_file = 'pytorch_roberta_sentiment.bin'
output_vocab_file = './'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')

All files saved
